In [77]:
import numpy as np
import pandas as pd
import os
import time
import glob
import torch
from torch.utils.data import TensorDataset, DataLoader
from torch.optim import lr_scheduler

from sklearn.preprocessing import StandardScaler
from tqdm import tqdm

from tools import EarlyStopping, adjust_learning_rate
import xLSTMTime

In [38]:
class Config():
    def __init__(self):
        self.seq_len = 9  # time length
        self.input_size = 10  # num_features
        self.target_len = 3
        self.head_size = 16
        self.num_heads = 2
        self.proj_factor_slstm = 4/3
        self.proj_factor_mlstm = 2
        self.layer_type = "msm"
        self.batch_first = True
        self.linear_embed_dim = 64
        self.split_ratio = 0.9
        self.batch_size = 32
        self.epochs = 500
        self.lr = 0.001
        self.pct_start = 0.3
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        self.lradj = 'TST'
        self.path = './results'
        self.patience = 100

configs = Config()

data

In [6]:
df = pd.read_csv('./dataset/train_target_assemble(18-21).csv')

In [8]:
# 시점 분리
df_data = df[df.columns[1:]]
df_stamp = df[df.columns[0]]

In [11]:
train_size=int(len(df_data) * configs.split_ratio)
train_set,test_set=df_data.iloc[:train_size],df_data.iloc[train_size:]
print(f"train size={len(train_set)}\ntest_size={len(test_set)}")

train size=129
test_size=15


In [14]:
scaled_columns=train_set.columns
scaler=StandardScaler()
scaler=scaler.fit(train_set[scaled_columns].values)
train_set[scaled_columns] = scaler.transform(train_set[scaled_columns].values)
test_set[scaled_columns] = scaler.transform(test_set[scaled_columns].values)

C:\Users\cbml\AppData\Local\Temp\ipykernel_28588\1719464611.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_set[scaled_columns] = scaler.transform(train_set[scaled_columns].values)
C:\Users\cbml\AppData\Local\Temp\ipykernel_28588\1719464611.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_set[scaled_columns] = scaler.transform(test_set[scaled_columns].values)


In [19]:
def create_dataset(dataset, seq_len=8, target_len=8):
    dataX, dataY = [], []
    for i in range(len(dataset) - seq_len - target_len):
        x_data = dataset.iloc[i:(i + seq_len), :].values
        dataX.append(x_data)
        y_data = dataset.iloc[(i + seq_len):(i + seq_len)+target_len, :].values
        dataY.append(y_data)
    return torch.Tensor(dataX), torch.Tensor(dataY)

In [20]:
X_train, y_train = create_dataset(train_set, configs.seq_len, configs.target_len)
X_test, y_test = create_dataset(test_set, configs.seq_len, configs.target_len)
print(X_train.shape, y_train.shape)

torch.Size([117, 9, 10]) torch.Size([117, 3, 10])


In [23]:
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=configs.batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=configs.batch_size, shuffle=False)

model

In [25]:
model = xLSTMTime.Model(configs)

In [ ]:
model.to(configs.device)

In [41]:
def validation(model, test_loader, configs):
    criterion = torch.nn.SmoothL1Loss()
    
    valid_losses = []
    model.eval()
    with torch.no_grad():
        for i, (inputs, targets) in enumerate(test_loader):
            inputs = inputs.to(configs.device)
            targets = targets.to(configs.device)

            outputs = model(inputs)

            outputs = outputs.detach().cpu()
            targets = targets.detach().cpu()

            loss = criterion(outputs, targets)
            valid_losses.append(loss)
    
    valid_losses = np.average(valid_losses)
    model.train()
    return valid_losses

In [42]:
def train_model(model, train_loader, test_loader, configs):
    criterion = torch.nn.SmoothL1Loss()
    optimizer = torch.optim.Adam(model.parameters(), lr=configs.lr)
    scheduler = lr_scheduler.OneCycleLR(optimizer = optimizer,
                                            steps_per_epoch = len(train_loader),
                                            pct_start = configs.pct_start,
                                            epochs = configs.epochs,
                                            max_lr = configs.lr)
    early_stopping = EarlyStopping(patience=configs.patience, verbose=True)


    for epoch in tqdm(range(configs.epochs), desc=f'Training', colour='GREEN'):
        iter_count = 0
        train_losses = []
        model.train()
        for i, (inputs, targets) in enumerate(train_loader):
            iter_count += 1

            optimizer.zero_grad()
            inputs = inputs.to(configs.device)
            targets = targets.to(configs.device)

            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            adjust_learning_rate(optimizer, scheduler, epoch + 1, configs, printout=False)
            scheduler.step()

            train_losses.append(loss.item())

            if (i + 1) % 100 == 0:
                print(f"\tepoch: {epoch+1} | iters: {i+1} | loss: {loss.item():.4f}")
                iter_count = 0

        train_losses = np.average(train_losses)
        valid_losses = validation(model, test_loader, configs)
        print(f"Epoch: {epoch+1}, Steps: {len(train_loader)} | Train Loss: {train_losses:.4f} Vali Loss: {valid_losses:.4f}") 
        early_stopping(valid_losses, model, configs.path)
        if early_stopping.early_stop:
            print("Early stopping")
            break
        
        adjust_learning_rate(optimizer, scheduler, epoch + 1, configs)

    best_model_path = configs.path + '/' + 'checkpoint.pth'
    model.load_state_dict(torch.load(best_model_path))

    return model

In [ ]:
trained_models = train_model(model, train_loader, test_loader, configs)

In [102]:
# predict

test_files = glob.glob('./dataset/test_*_assemble.csv')

result_df = pd.DataFrame()

for i, files in tqdm(enumerate(test_files)):
    test_df = pd.read_csv(files)
    test_df = test_df[test_df.columns[1:]]
    
    scaled_test = scaler.transform(test_df.values)

    X_predict = torch.Tensor(scaled_test)
    pred_dataset = TensorDataset(X_predict)
    pred_loader = DataLoader(pred_dataset, batch_size=configs.batch_size, shuffle=False)


    trained_models.eval()
    with torch.no_grad():
        for inputs in pred_loader:
            inputs = inputs[0]
            inputs = inputs.unsqueeze(dim=0)
            inputs = inputs.to(configs.device)

            outputs = trained_models(inputs)

            outputs = outputs.detach().cpu()

    df = pd.DataFrame(outputs.squeeze(0).numpy())
    unscaled_df = pd.DataFrame(scaler.inverse_transform(df), columns=df_data.columns)
    result_df = pd.concat([result_df, unscaled_df], ignore_index=True)

25it [00:06,  3.82it/s]


In [109]:
result_df

,건고추,사과,감자,배,깐마늘,무,상추,배추,양파,대파
0,609302.1875,29254.253906,36617.507812,36361.433594,167478.250000,32615.095703,2420.249756,19262.695312,1213.200684,1894.626587
1,617023.6875,28497.154297,36231.136719,36630.433594,163629.078125,31210.527344,2224.104248,16854.636719,1183.804077,1786.938843
2,605248.8125,28304.480469,37114.960938,37284.652344,163605.156250,28861.509766,2075.899170,16491.835938,1200.915039,1810.254028
3,552616.6250,26410.064453,47353.078125,37105.437500,161307.875000,11116.015625,1004.247131,8085.214355,374.345734,1188.465088
4,554617.5000,26183.451172,47792.765625,37169.054688,157573.437500,11420.889648,1005.006531,8084.312012,412.889282,1237.102417
...,...,...,...,...,...,...,...,...,...,...
70,583119.3750,30747.593750,36373.832031,41552.609375,166326.140625,20129.474609,2590.409424,21046.162109,1303.507935,2133.342041
71,582978.1250,30755.613281,36491.417969,41536.640625,167206.984375,19402.986328,2463.458008,19998.230469,1374.092163,2130.970459
72,551106.0000,27032.623047,59036.109375,39335.644531,163086.734375,10135.536133,900.185852,8635.014648,470.986206,1279.673218
73,551215.8125,26565.105469,60595.613281,38563.085938,159367.765625,10575.388672,851.039246,8842.406250,489.761078,1286.642944


In [118]:
result_df.rename(columns={'깐마늘':'깐마늘(국산)'}, inplace=True)

In [130]:
submission = pd.read_csv('./dataset/sample_submission.csv')

In [131]:
submission

,시점,감자,건고추,깐마늘(국산),대파,무,배추,사과,상추,양파,배
0,TEST_00+1순,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,TEST_00+2순,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,TEST_00+3순,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,TEST_01+1순,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,TEST_01+2순,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
70,TEST_23+2순,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
71,TEST_23+3순,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
72,TEST_24+1순,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
73,TEST_24+2순,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [132]:
sub_date = submission[submission.columns[0]]
sub_data = submission[submission.columns[1:]]

In [133]:
result_df = result_df[sub_data.columns]

In [120]:
result_df

,감자,건고추,깐마늘(국산),대파,무,배추,사과,상추,양파,배
0,36617.507812,609302.1875,167478.250000,1894.626587,32615.095703,19262.695312,29254.253906,2420.249756,1213.200684,36361.433594
1,36231.136719,617023.6875,163629.078125,1786.938843,31210.527344,16854.636719,28497.154297,2224.104248,1183.804077,36630.433594
2,37114.960938,605248.8125,163605.156250,1810.254028,28861.509766,16491.835938,28304.480469,2075.899170,1200.915039,37284.652344
3,47353.078125,552616.6250,161307.875000,1188.465088,11116.015625,8085.214355,26410.064453,1004.247131,374.345734,37105.437500
4,47792.765625,554617.5000,157573.437500,1237.102417,11420.889648,8084.312012,26183.451172,1005.006531,412.889282,37169.054688
...,...,...,...,...,...,...,...,...,...,...
70,36373.832031,583119.3750,166326.140625,2133.342041,20129.474609,21046.162109,30747.593750,2590.409424,1303.507935,41552.609375
71,36491.417969,582978.1250,167206.984375,2130.970459,19402.986328,19998.230469,30755.613281,2463.458008,1374.092163,41536.640625
72,59036.109375,551106.0000,163086.734375,1279.673218,10135.536133,8635.014648,27032.623047,900.185852,470.986206,39335.644531
73,60595.613281,551215.8125,159367.765625,1286.642944,10575.388672,8842.406250,26565.105469,851.039246,489.761078,38563.085938


In [134]:
my_result = pd.concat([sub_date, result_df], axis=1)

In [135]:
my_result

,시점,감자,건고추,깐마늘(국산),대파,무,배추,사과,상추,양파,배
0,TEST_00+1순,36617.507812,609302.1875,167478.250000,1894.626587,32615.095703,19262.695312,29254.253906,2420.249756,1213.200684,36361.433594
1,TEST_00+2순,36231.136719,617023.6875,163629.078125,1786.938843,31210.527344,16854.636719,28497.154297,2224.104248,1183.804077,36630.433594
2,TEST_00+3순,37114.960938,605248.8125,163605.156250,1810.254028,28861.509766,16491.835938,28304.480469,2075.899170,1200.915039,37284.652344
3,TEST_01+1순,47353.078125,552616.6250,161307.875000,1188.465088,11116.015625,8085.214355,26410.064453,1004.247131,374.345734,37105.437500
4,TEST_01+2순,47792.765625,554617.5000,157573.437500,1237.102417,11420.889648,8084.312012,26183.451172,1005.006531,412.889282,37169.054688
...,...,...,...,...,...,...,...,...,...,...,...
70,TEST_23+2순,36373.832031,583119.3750,166326.140625,2133.342041,20129.474609,21046.162109,30747.593750,2590.409424,1303.507935,41552.609375
71,TEST_23+3순,36491.417969,582978.1250,167206.984375,2130.970459,19402.986328,19998.230469,30755.613281,2463.458008,1374.092163,41536.640625
72,TEST_24+1순,59036.109375,551106.0000,163086.734375,1279.673218,10135.536133,8635.014648,27032.623047,900.185852,470.986206,39335.644531
73,TEST_24+2순,60595.613281,551215.8125,159367.765625,1286.642944,10575.388672,8842.406250,26565.105469,851.039246,489.761078,38563.085938


In [138]:
my_result.to_csv(f'sample_submission.csv', index=False, encoding="utf-8-sig")